# Treinamento do modelo usando Random Forest no SageMaker

### Importando bibliotecas e variáveis de ambiente 

In [ ]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Carregar variáveis de ambiente a partir de um arquivo .env
load_dotenv()

# Definir a região a ser usada (us-east-1)
region_name = 'us-east-1'

# Carregar variáveis de ambiente para conectar ao banco de dados RDS
database_name = os.getenv('DATABASE_NAME')
table_name = os.getenv('TABLE_NAME')
rds_host = os.getenv('RDS_HOST')
rds_port = os.getenv('RDS_PORT')
rds_user = os.getenv('RDS_USER')
rds_password = os.getenv('RDS_PASSWORD')

# Criar uma sessão boto3
boto3_session = boto3.Session()

# Criar um cliente S3 usando a sessão boto3
s3_client = boto3_session.client('s3')

# Criar um cliente SageMaker usando a sessão boto3
sm_boto3 = boto3_session.client('sagemaker')

# Criar uma sessão SageMaker
sess = sagemaker.Session(boto_session=boto3_session)

# Obter a região da sessão boto3
region = sess.boto_session.region_name

# Definir o nome do bucket S3 a partir da variável de ambiente
bucket = os.getenv('BUCKET_NAME')

# Imprimir o nome do bucket que está sendo usado
print("Usando o bucket " + bucket)


### Importando a database do RDS

In [ ]:
# Criar a string de conexão para o SQLAlchemy
db_url = f'mysql+pymysql://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{database_name}'

# Conectar-se ao banco de dados
engine = create_engine(db_url)

# Consultar o banco de dados e carregar os dados em um DataFrame
query = f'SELECT * FROM {table_name}'
hotel_reservas = pd.read_sql(query, con=engine)
hotel_reservas

### Tratamento de dados

In [ ]:
# Remover as colunas que não serão utilizadas na análise ou no modelo
hotel_reservas.drop(['Booking_ID','booking_status', 'no_of_previous_bookings_not_canceled','repeated_guest', 'no_of_previous_cancellations'], axis=1, inplace=True)

# Converter as colunas categóricas em variáveis dummy (one-hot encoding)
hotel_reservas = pd.get_dummies(hotel_reservas, prefix=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'],
                                columns=['type_of_meal_plan', 'room_type_reserved', 'market_segment_type'])

# Exibir o DataFrame transformado
hotel_reservas

In [ ]:
# Obter a lista de todas as colunas do DataFrame hotel_reservas
features = list(hotel_reservas.columns)

# Exibir a lista de features (nomes das colunas)
features

In [ ]:
# Remover e obter o décimo primeiro elemento da lista de features (coluna que será usada como label)
label = features.pop(10)

# Exibir o nome da coluna que foi removida da lista de features e armazenada em label
label

In [ ]:
# Selecionar as colunas restantes como variáveis independentes (features)
x = hotel_reservas[features]

# Selecionar a coluna removida anteriormente como variável dependente (label)
y = hotel_reservas[label]

In [ ]:
# Exibir as primeiras cinco linhas do DataFrame x (variáveis independentes)
x.head()

In [ ]:
# Exibir as primeiras cinco linhas do DataFrame y (variáveis independentes)
y.head()

In [ ]:
# Exibir a forma (número de linhas e colunas) do DataFrame x (variáveis independentes)
x.shape

In [ ]:
# Exibir a contagem de valores únicos na série y (variável dependente)
y.value_counts()

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [ ]:
# Exibir a forma (número de linhas e colunas) do conjunto de treinamento das variáveis independentes
print(X_train.shape)

# Exibir a forma (número de linhas e colunas) do conjunto de teste das variáveis independentes
print(X_test.shape)

# Exibir a forma (número de linhas) do conjunto de treinamento da variável dependente
print(y_train.shape)

# Exibir a forma (número de linhas) do conjunto de teste da variável dependente
print(y_test.shape)

In [ ]:
# Converter o conjunto de treinamento das variáveis independentes para um DataFrame
trainX = pd.DataFrame(X_train)

# Adicionar a variável dependente ao conjunto de treinamento
trainX[label] = y_train

# Converter o conjunto de teste das variáveis independentes para um DataFrame
testX = pd.DataFrame(X_test)

# Adicionar a variável dependente ao conjunto de teste
testX[label] = y_test

In [ ]:
# Exibir a forma (número de linhas e colunas) do DataFrame de treinamento completo (variáveis independentes + dependente)
print(trainX.shape)

# Exibir a forma (número de linhas e colunas) do DataFrame de teste completo (variáveis independentes + dependente)
print(testX.shape)

In [ ]:
# Salvar o DataFrame de treinamento completo em um arquivo CSV chamado "train-H.csv"
trainX.to_csv("train-H.csv", index=False)

# Salvar o DataFrame de teste completo em um arquivo CSV chamado "test-H.csv"
testX.to_csv("test-H.csv", index=False)

In [ ]:
# Enviar dados para o S3. O SageMaker irá utilizar os dados de treinamento do S3
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"

# upload do arquivo CSV de treinamento para o bucket S3
trainpath = sess.upload_data(
    path="train-H.csv", bucket=bucket, key_prefix=sk_prefix
)

# upload do arquivo CSV de teste para o bucket S3
testpath = sess.upload_data(
    path="test-H.csv", bucket=bucket, key_prefix=sk_prefix
)

# Imprimir o caminho completo do arquivo de treinamento no S3
print(trainpath)

# Imprimir o caminho completo do arquivo de teste no S3
print(testpath)

### Criação do script para o SageMaker

In [ ]:
%%writefile scriptrf.py

# Importar bibliotecas necessárias
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import boto3
import os
import argparse
import pandas as pd
from dotenv import load_dotenv

# Carregar variáveis de ambiente
bucket = os.getenv('BUCKET_NAME')

# Função para carregar o modelo a partir de um diretório
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":

    # Extraindo argumentos
    parser = argparse.ArgumentParser()

    # Hiperparâmetros enviados pelo cliente são passados como argumentos de linha de comando para o script
    parser.add_argument("--n_estimators", type=int, default=100)
    
    # Diretórios de dados, modelo e saída
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-H.csv")
    parser.add_argument("--test-file", type=str, default="test-H.csv")

    args, _ = parser.parse_known_args()
    
    # Lendo dados
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    # Obter a lista de features e o label
    features = list(train_df.columns)
    label = features.pop(-1)
    
    # Construir datasets de treinamento e teste
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]
  
    # Treinar o modelo RandomForest
    model = RandomForestClassifier(n_estimators=args.n_estimators, verbose=3, n_jobs=-1)
    model.fit(X_train, y_train)
    
    # Salvar o modelo treinado
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    
    # Enviar o modelo treinado para o bucket S3
    s3 = boto3.client('s3')
    s3.upload_file(model_path, bucket, "model.joblib")
    
    # Fazer previsões e avaliar o modelo
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    # Imprimir a acurácia do modelo e o relatório de teste
    print('Model Accuracy is: ', test_acc)
    print('Testing Report: ')
    print(test_rep)

### Configuração do SageMaker

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

# Definir a versão do framework a ser usada
FRAMEWORK_VERSION = "1.2-1"

# Obter o nome do bucket a partir da variável de ambiente
bucket = os.getenv('BUCKET_NAME')

# Criar um estimador SKLearn para treinar o modelo no SageMaker
sklearn_estimator = SKLearn(
    entry_point="scriptrf.py",  # Script de entrada para treinar o modelo
    role=os.getenv('SAGEMAKER_ROLE'),  # Função do IAM para execução do SageMaker
    instance_count=1,  # Número de instâncias a serem usadas para treinamento
    instance_type="ml.m5.large",  # Tipo de instância a ser usada para treinamento
    framework_version=FRAMEWORK_VERSION,  # Versão do framework SKLearn
    base_job_name="RF-custom-sklearn",  # Nome base para o trabalho de treinamento
    hyperparameters={
        "n_estimators": 100,  # Hiperparâmetro para o número de estimadores do RandomForest
    },
    use_spot_instances=True,  # Usar instâncias spot para reduzir custos
    max_wait=7200,  # Tempo máximo de espera em segundos para instâncias spot
    max_run=3600,  # Tempo máximo de execução em segundos para o trabalho de treinamento
    sagemaker_session=sess  # Sessão do SageMaker a ser usada
)

### Treinamento, previsões e avaliação

In [ ]:
# Iniciar o trabalho de treinamento usando o estimador SKLearn
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)